In [1]:
import socket  # 소켓 프로그래밍
import json    # JSON 데이터 처리
import threading  # 다중 클라이언트 처리를 위한 스레딩

HOST = '0.0.0.0'   # 모든 IP로부터 접속 허용
PORT = 12000       # 포트 번호
BUFFER_SIZE = 2048 # 데이터 수신 크기

# 메시지를 처리하는 함수: 옵션에 따라 다르게 응답
def process_message(username, message, option_raw):
    try:
        if "." in str(option_raw):
            raise ValueError("float")  # 실수 예외 처리

        option = int(option_raw)

        if option < 1:  # 1 미만
            return {'status': 401, 'error': 'option input error: integer less than 1'}, False
        elif option > 4:  # 4 초과
            return {'status': 403, 'error': 'option input error: greater than 4'}, False

    except ValueError as ve:
        if str(ve) == "float":  # 실수
            return {'status': 402, 'error': 'option input error: non-integer'}, False
        else:  # 문자 등 기타
            return {'status': 405, 'error': 'option input error: non-numeric input'}, False

    # 실제 메시지 처리
    if option == 1:  # 그대로
        result = message
    elif option == 2:  # 대문자
        result = message.upper()
    elif option == 3:  # 소문자
        result = message.lower()

    return {'status': 200, 'message': f'[{username}]: {result}'}, False

# 서버 실행 함수
def start_server():
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as server_socket:
        server_socket.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)  # 포트 재사용
        server_socket.bind((HOST, PORT))
        server_socket.listen(5)
        print(f"[SERVER] Listening on port {PORT}...")

        while True:
            client_socket, addr = server_socket.accept()
            with client_socket:
                data = client_socket.recv(BUFFER_SIZE).decode()

                try:
                    request = json.loads(data)
                    username = request.get('user')
                    message = request.get('message')
                    option = request.get('echooption')

                    response, _ = process_message(username, message, option)
                    client_socket.sendall(json.dumps(response).encode())

                    print(f"[RECV] {request}")
                    print(f"[SEND] {response}")

                except Exception as e:
                    error_response = json.dumps({"status": 500, "error": "Invalid request format"})
                    client_socket.sendall(error_response.encode())
                    print(f"[ERROR] {e}")

# 서버를 백그라운드 스레드로 실행
server_thread = threading.Thread(target=start_server, daemon=True)
server_thread.start()

[SERVER] Listening on port 12000...


In [2]:
import socket
import json

SERVER_HOST = '127.0.0.1'
SERVER_PORT = 12000
BUFFER_SIZE = 2048

print("[CLIENT] 연결을 시작합니다.")
username = input("사용자 이름: ")

while True:
    option_input = input("옵션 (1: 그대로, 2: 대문자, 3: 소문자, 4: 종료): ")

    # 💡 옵션 4 입력 시 패킷 전송 없이 종료
    if option_input.strip() == "4":
        print("[CLIENT] 종료 요청.")
        break

    message = input("메시지 입력: ") 

    packet = {
        "user": username,
        "echooption": option_input,
        "message": message
    }

    try:
        with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as client_socket:
            client_socket.connect((SERVER_HOST, SERVER_PORT))
            client_socket.sendall(json.dumps(packet).encode())

            response = client_socket.recv(BUFFER_SIZE).decode()
            response_data = json.loads(response)

            # 정상 메시지 출력
            if "message" in response_data:
                print("응답:", response_data["message"], ", status:", response_data["status"])
            # 오류 메시지 출력
            elif "error" in response_data:
                print(f"오류 {response_data['status']}: {response_data['error']}")

    except Exception as e:
        print(f"[ERROR] 서버 연결 실패: {e}")

[CLIENT] 연결을 시작합니다.
응답: [김현진]: QWER , status: 200
응답: [김현진]: qwer , status: 200
응답: [김현진]: qwer , status: 200
[CLIENT] 종료 요청. 서버에 패킷을 보내지 않고 종료합니다.


[RECV] {'user': '김현진', 'echooption': '2', 'message': 'qwer'}
[SEND] {'status': 200, 'message': '[김현진]: QWER'}
[RECV] {'user': '김현진', 'echooption': '3', 'message': 'QWER'}
[SEND] {'status': 200, 'message': '[김현진]: qwer'}
[RECV] {'user': '김현진', 'echooption': '1', 'message': 'qwer'}
[SEND] {'status': 200, 'message': '[김현진]: qwer'}
